LIBRARY IMPORT

In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
import os

READING FILE

In [3]:
train_review =[]
for line in open('../movie-sentiment-analysis/aclImdb/movie_data/full_train.txt','r',encoding='utf-8'):
    train_review.append(line.strip())

In [4]:
test_review=[]
for line in open('../movie-sentiment-analysis/aclImdb/movie_data/full_test.txt','r',encoding='utf-8'):
    test_review.append(line.strip())

In [5]:
train_review[1]

'Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they\'ll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it\'s like to be homeless? That is Goddard Bolt\'s lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days withou

CLEANING AND PROCESSING

In [7]:
no_space = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
with_space = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def cleaningandprocessing(reviews):
    reviews = [no_space.sub("",line.lower()) for line in reviews]
    reviews = [with_space.sub(" ",line) for line in reviews]
    return reviews
clean_train_reviews = cleaningandprocessing(train_review)
clean_test_reviews = cleaningandprocessing(test_review)

In [8]:
clean_train_reviews[1]

'homelessness or houselessness as george carlin stated has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or vote for the matter most people think of the homeless as just a lost cause while worrying about things such as racism the war on iraq pressuring kids to succeed technology the elections inflation or worrying if theyll be next to end up on the streets but what if you were given a bet to live on the streets for a month without the luxuries you once had from a home the entertainment sets a bathroom pictures on the wall a computer and everything you once treasure to see what its like to be homeless that is goddard bolts lesson mel brooks who directs who stars as bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival jeffery tambor to see if he can live in the streets for thirty days without the luxuries if bolt succeeds he can do what he w

VECTORIZATION

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(binary=True)
cv.fit(clean_train_reviews)
X = cv.transform(clean_train_reviews)
X_test = cv.transform(clean_test_reviews)

TRAIN TEST SPLIT

In [14]:
from sklearn.model_selection import train_test_split
target = [1 if i <12500 else 0 for i in range(25000)]
x_train, x_test, y_train, y_test = train_test_split(X, target, test_size = 0.75, random_state=42)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

for c in [0.01,0.05,0.1,0.25,0.5,1]:
    model1 = LogisticRegression(C=c)
    model1.fit(x_train,y_train)
    predictions = model1.predict(x_test)
    print("For ",c)
    print("\n")
    print("Confusion Matrix: ",confusion_matrix(y_test, predictions))
    print("\n")
    print("Classification Report: ", classification_report(y_test, predictions))
    print("\n")
    print("Accuracy Score: ",accuracy_score(y_test, predictions)*100)
    print("\n")
    print("\n")

For  0.01


Confusion Matrix:  [[7894 1434]
 [1306 8116]]


Classification Report:               precision    recall  f1-score   support

          0       0.86      0.85      0.85      9328
          1       0.85      0.86      0.86      9422

avg / total       0.85      0.85      0.85     18750



Accuracy Score:  85.38666666666667




For  0.05


Confusion Matrix:  [[7980 1348]
 [1251 8171]]


Classification Report:               precision    recall  f1-score   support

          0       0.86      0.86      0.86      9328
          1       0.86      0.87      0.86      9422

avg / total       0.86      0.86      0.86     18750



Accuracy Score:  86.13866666666667




For  0.1


Confusion Matrix:  [[7988 1340]
 [1251 8171]]


Classification Report:               precision    recall  f1-score   support

          0       0.86      0.86      0.86      9328
          1       0.86      0.87      0.86      9422

avg / total       0.86      0.86      0.86     18750



Accuracy Score:  86.

ACCURACY SCORE FOR 0.1 IS 86.18 MAX

In [17]:
final_model = LogisticRegression(C=0.1)
final_model.fit(X, target)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
predictions1 = final_model.predict(X_test)

In [19]:
print("Accuracy score is: ",accuracy_score(target,predictions1))
print("\n")
print("Confusion matrix is: ", confusion_matrix(target, predictions1))
print("\n")
print("Classification matrix is: ",classification_report(target, predictions1))

Accuracy score is:  0.87932


Confusion matrix is:  [[10980  1520]
 [ 1497 11003]]


Classification matrix is:               precision    recall  f1-score   support

          0       0.88      0.88      0.88     12500
          1       0.88      0.88      0.88     12500

avg / total       0.88      0.88      0.88     25000



FINAL ACCURACY IS 87.93

ANALYSING SMALLEST AND LARGEST COEFFICIENT

In [20]:
featurecoeff = {
    word: coef for word, coef in zip(
        cv.get_feature_names(),  final_model.coef_[0]
    )
}

POSITIVE MODE

In [24]:
for bestpositiveword in sorted(featurecoeff.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print(bestpositiveword)

('excellent', 1.0462425726513722)
('perfect', 0.9011123882566677)
('superb', 0.7412650434568396)
('refreshing', 0.7346347629237331)
('wonderfully', 0.7208587539775335)
('great', 0.7155906390484156)
('enjoyable', 0.708595108720496)
('rare', 0.7043862298777142)
('amazing', 0.6853421284235471)
('incredible', 0.6815248358129178)


In [25]:
for bestnegativeword in sorted(featurecoeff.items(), key = lambda x: x[1])[:10]:
    print(bestnegativeword)

('worst', -1.5501289372124922)
('waste', -1.384471585638481)
('awful', -1.1764353144994475)
('poorly', -1.0913709000198755)
('disappointment', -1.073590069926218)
('boring', -0.9841817187621853)
('dull', -0.8952781348904849)
('disappointing', -0.8919646656110167)
('lacks', -0.8323810576904175)
('mess', -0.8085272903145483)
